In [1]:
import random
from datetime import datetime, timedelta
from PyQt5.QtCore import QTimer

def generate_reset_code(email, parent_widget=None):
    try:
        connection = create_connection()
        if connection:
            print("Connection successful")
        else:
            print("Connection failed")

        cursor = connection.cursor()

        # Check if email exists
        cursor.execute("SELECT * FROM userlist WHERE email = %s", (email,))
        result = cursor.fetchone()

        if not result:
            QMessageBox.warning(parent_widget, "Error", "Email does not exist.")
            return

        # Generate a random 6-digit reset code
        reset_code = str(random.randint(100000, 999999))
        current_time = datetime.now()

        # Save the reset code and timestamp in the database
        query = "UPDATE userlist SET reset_code = %s, code_created_at = %s WHERE email = %s"
        print(f"Executing query: {query} with values: {reset_code}, {current_time}, {email}")
        cursor.execute(query, (reset_code, current_time, email))
        connection.commit()

#         QMessageBox.information(parent_widget, "Code Generated", f"Reset code sent: {reset_code}")

        # Start a timer to clear the reset code after 15 seconds
        timer = QTimer()
        timer.singleShot(60000, lambda: clear_reset_code(email))

    except mysql.connector.Error as error:
        QMessageBox.critical(parent_widget, "Database Error", f"Failed to generate reset code: {error}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
            
    return reset_code

def clear_reset_code(email, parent_widget=None):
    """Clear the reset code after 15 seconds."""
    try:
        connection = create_connection()
        cursor = connection.cursor()

        # Clear the reset code and timestamp
        query = "UPDATE userlist SET reset_code = NULL, code_created_at = NULL WHERE email = %s"
        cursor.execute(query, (email,))
        connection.commit()
        print(f"Reset code cleared for email: {email}")

    except mysql.connector.Error as error:
        print(f"Failed to clear reset code: {error}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
            
def verify_reset_code(email, entered_code, parent_widget=None):
    try:
        connection = create_connection()
        cursor = connection.cursor()

        # Fetch the reset code and creation timestamp
        query = "SELECT reset_code, code_created_at FROM userlist WHERE email = %s"
        cursor.execute(query, (email,))
        result = cursor.fetchone()

        if not result or not result[0]:  # No reset code exists
            QMessageBox.warning(parent_widget, "Error", "No reset code found or it has expired.")
            return False

        reset_code, code_created_at = result
        if reset_code != entered_code:
            QMessageBox.warning(parent_widget, "Error", "Reset code does not match.")
            return False

        # Check if the code has expired (older than 15 seconds)
        current_time = datetime.now()
        if (current_time - code_created_at).total_seconds() > 60:
            QMessageBox.warning(parent_widget, "Error", "Reset code has expired.")
            return False

        # If the reset code matches and is valid
        QMessageBox.information(parent_widget, "Success", "Reset code verified. You may now reset your password.")
        return True

    except mysql.connector.Error as error:
        QMessageBox.critical(parent_widget, "Database Error", f"Failed to verify reset code: {error}")
        return False
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [2]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import random
import time

# # Generate a 6-digit random code
# reset_code = random.randint(100000, 999999)

# # Token expiration time (e.g., 15 minutes)
# expires_at = time.time() + 900  # 15 minutes from now

def send_reset_email(email, parent_widget=None):
    # Email configuration
    sender_email = "zubairmahnoor11@gmail.com"  # Replace with your email
    sender_password = "qpom jfvb vlrw sgbk"  # Replace with your email password or app password
    subject = "Your Password Reset Code"
    reset_code = generate_reset_code(email, parent_widget)
    body = f"Your password reset code is: {reset_code}\nThis code will expire in 15 minutes."

    # Create the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Connect to the SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Use the SMTP server and port of your email provider
        server.starttls()  # Upgrade the connection to a secure encrypted SSL/TLS connection
        server.login(sender_email, sender_password)  # Log in to your email account
        server.send_message(msg)  # Send the email
        server.quit()  # Disconnect from the server
        return True
    except Exception as e:
        print(f"Failed to send email: {e}")
        return False

In [3]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLineEdit, QPushButton, QLabel, 
    QVBoxLayout, QHBoxLayout, QFormLayout, QMessageBox
)
from PyQt5.QtGui import QFont
from PyQt5.QtCore import Qt
import mysql.connector
from PyQt5.QtWidgets import QInputDialog
import os
import threading
import subprocess
import re

# Database connection
def create_connection():
    return mysql.connector.connect(
        host="localhost",  
        user="root",  
        password="",  
        database="viz_optilytics"  
    )


class SignupForm(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Signup Form')
        self.setGeometry(200, 200, 600, 400)  # Window size
        self.setStyleSheet("background-color: #2D2D2D;")  # Dark background

        # Font for title
        title_font = QFont('Georgia', 28, QFont.Bold | QFont.StyleItalic)  # New font style for the title
        
        # Viz Optilytics Title
        title_label = QLabel('Viz Optilytics')
        title_label.setFont(title_font)
        title_label.setStyleSheet('color: #E5B131;')  # Gold color
        title_label.setAlignment(Qt.AlignCenter)

        # Layout for form fields
        form_layout = QVBoxLayout()
        form_layout.setContentsMargins(40, 10, 40, 20)  # Set margins around the window (left, top, right, bottom)

        # Create a function to add each row (label + input field)
        def add_row(label_text, placeholder_text):
            # Horizontal layout for label and field
            row_layout = QHBoxLayout()

            # Label
            label = QLabel(label_text)
            label.setAlignment(Qt.AlignCenter)
            label.setStyleSheet("""
                
                color: white;
                border-radius: 10px;
                padding: 10px 20px;
                font-size: 14px;
                font-weight: bold;
            """)
            label.setFixedWidth(160)  # Increased width for label box to prevent text overflow

            # Input field
            input_field = QLineEdit()
            input_field.setPlaceholderText(placeholder_text)
            input_field.setStyleSheet("""
                background-color: #606060;
                color: white;
                border-radius: 10px;
                padding: 10px;
            """)
            input_field.setFixedHeight(40)

            # Add spacing between label and input
            row_layout.addWidget(label)
            row_layout.addSpacing(20)  # Add space between label and input field
            row_layout.addWidget(input_field)

            # Add the row layout to the main form layout
            form_layout.addLayout(row_layout)
            return input_field

        # Create each row
        self.username_input = add_row('UserName', 'Testing123')
        self.email_input = add_row('Email', 'Testing@gmail.com')
        self.password_input = add_row('Password', '******')
        self.password_input.setEchoMode(QLineEdit.Password)
        self.verify_password_input = add_row('Verify Password', '******')
        self.verify_password_input.setEchoMode(QLineEdit.Password)

        # Add some vertical spacing before the Register button
        form_layout.addSpacing(30)  # Adds space before the Register button

        # Register Button
        register_button = QPushButton('Register Now')
        register_button.setStyleSheet("""
            QPushButton {
                background-color: #00A74D;  /* Green color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #009945;
            }
            QPushButton:pressed {
                background-color: #008838;
            }
        """)
        register_button.setFixedWidth(250)  # Increase button width to match input fields
        register_button.clicked.connect(self.handle_signup)

        # Add Register button at the end
        form_layout.addWidget(register_button)
        form_layout.setAlignment(register_button, Qt.AlignCenter)

        # Add "Already have an account?" text
        already_account_label = QLabel('Already have an account?')
        already_account_label.setStyleSheet('color: white; font-size: 12px;')
        already_account_label.setAlignment(Qt.AlignCenter)

        # Login Button
        self.login_button = QPushButton('Login Now')  # Correctly store the button as an instance variable
        self.login_button.setStyleSheet("""
            QPushButton {
                background-color: #d40d0d;  /* Red color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #C40C0C;
            }
            QPushButton:pressed {
                background-color: #b50b0b;
            }
        """)
        self.login_button.setFixedWidth(250)
        self.login_button.clicked.connect(self.open_login)  # Connect the signal properly

        # Add "Already have an account?" text and Login button to form layout
        form_layout.addSpacing(15)  # Add some space before the "Already have an account?" text
        form_layout.addWidget(already_account_label)
        form_layout.setAlignment(already_account_label, Qt.AlignCenter)
        form_layout.addWidget(self.login_button)  # Add self.login_button here
        form_layout.setAlignment(self.login_button, Qt.AlignCenter)

        # Main Layout
        main_layout = QVBoxLayout()
        main_layout.setSpacing(10)  # Reduce spacing between title and form
        main_layout.addWidget(title_label)
        main_layout.addLayout(form_layout)

        self.setLayout(main_layout)

    def open_login(self):
        self.close()  # Close the signup form
        self.login_window = LoginForm()  # Assuming LoginForm is the class for the login page
        self.login_window.show()  # Show the login window

    def handle_signup(self):
        email = self.email_input.text()
        username = self.username_input.text()
        password = self.password_input.text()
        verify_password = self.verify_password_input.text()

        # Check if all fields are filled
        if not email or not username or not password or not verify_password:
            QMessageBox.warning(self, "Input Error", "All fields are required.")
            return

        # Validate email, username, and password
        if not self.validate_email(email):
            QMessageBox.warning(self, "Invalid Email", "Please enter a valid email address.")
            return

        if not self.validate_username(username):
            QMessageBox.warning(self, "Invalid Username", "Please enter a valid username.")
            return

        if not self.validate_password(password):
            QMessageBox.warning(self, "Invalid Password", "Password does not meet the criteria.")
            return

        # Check if password and verify password match
        if password != verify_password:
            QMessageBox.warning(self, "Password Error", "Passwords do not match.")
            return

        try:
            connection = create_connection()
            cursor = connection.cursor()

            # Check if email or username already exists
            cursor.execute("SELECT * FROM userlist WHERE email = %s OR username = %s", (email, username))
            result = cursor.fetchone()

            if result:
                QMessageBox.warning(self, "Error", "Email or Username already exists.")
                return

            # Insert into database
            query = "INSERT INTO userlist (email, username, password) VALUES (%s, %s, %s)"
            cursor.execute(query, (email, username, password))
            connection.commit()
            QMessageBox.information(self, "Success", "Signup successful!")
            self.clear_fields()

        except mysql.connector.Error as error:
            QMessageBox.critical(self, "Database Error", f"Failed to insert data into MySQL: {error}")
        finally:
            if cursor:
                cursor.close()
            if connection:
                connection.close()

    def clear_fields(self):
        self.email_input.clear()
        self.username_input.clear()
        self.password_input.clear()
        self.verify_password_input.clear()

    def validate_email(self, email):
        pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
        return re.match(pattern, email)

    def validate_password(self, password):
        if len(password) < 8:
            return False
        if not re.search(r"[A-Z]", password):
            return False
        if not re.search(r"[a-z]", password):
            return False
        if not re.search(r"[0-9]", password):
            return False
        if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", password):
            return False
        return True

    def validate_username(self, username):
        if len(username) < 3 or len(username) > 15:
            return False
        if not re.match(r'^\w+$', username):  # Only letters, numbers, and underscores allowed
            return False
        return True




    
class LoginForm(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Login Form')
        self.setGeometry(200, 200, 600, 400)  # Window size
        self.setStyleSheet("background-color: #2D2D2D;")  # Dark background

        # Font for title
        title_font = QFont('Georgia', 28, QFont.Bold | QFont.StyleItalic)  # New font style for the title
        
        # Viz Optilytics Title
        title_label = QLabel('Viz Optilytics')
        title_label.setFont(title_font)
        title_label.setStyleSheet('color: #E5B131;')  # Gold color
        title_label.setAlignment(Qt.AlignCenter)

        # Layout for form fields
        form_layout = QVBoxLayout()
        form_layout.setContentsMargins(40, 10, 40, 20)  # Set margins around the window (left, top, right, bottom)

        # Create a function to add each row (label + input field)
        def add_row(label_text, placeholder_text):
            # Horizontal layout for label and field
            row_layout = QHBoxLayout()

            # Label
            label = QLabel(label_text)
            label.setAlignment(Qt.AlignCenter)
            label.setStyleSheet("""
                
                color: white;
                border-radius: 10px;
                padding: 10px 20px;
                font-size: 14px;
                font-weight: bold;
            """)
            label.setFixedWidth(160)  # Increased width for label box to prevent text overflow

            # Input field
            input_field = QLineEdit()
            input_field.setPlaceholderText(placeholder_text)
            input_field.setStyleSheet("""
                background-color: #606060;
                color: white;
                border-radius: 10px;
                padding: 10px;
            """)
            input_field.setFixedHeight(40)

            # Add spacing between label and input
            row_layout.addWidget(label)
            row_layout.addSpacing(20)  # Add space between label and input field
            row_layout.addWidget(input_field)

            # Add the row layout to the main form layout
            form_layout.addLayout(row_layout)
            return input_field

        # Create each row
        self.email_input = add_row('Email', 'Testing@gmail.com')
        self.password_input = add_row('Password', '******')
        self.password_input.setEchoMode(QLineEdit.Password)

        # Add some vertical spacing before the Register button
        form_layout.addSpacing(30)  # Adds space before the Register button

        # Register Button
        login_button = QPushButton('Login Now')
        login_button.setStyleSheet("""
            QPushButton {
                background-color: #00A74D;  /* Green color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #009945;
            }
            QPushButton:pressed {
                background-color: #008838;
            }
        """)
        login_button.setFixedWidth(250)  # Increase button width to match input fields
        login_button.clicked.connect(self.handle_login)

        # Add Register button at the end
        form_layout.addWidget(login_button)
        form_layout.setAlignment(login_button, Qt.AlignCenter)

        # Login Button
        forgot_password = QPushButton('forget Password')
        forgot_password.setStyleSheet("""
            QPushButton {
                background-color: #d40d0d;  /* Green color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #C40C0C;
            }
            QPushButton:pressed {
                background-color: #b50b0b;
            }
        """)
        forgot_password.setFixedWidth(250)
        forgot_password.clicked.connect(self.reset_password)

        # Add "Already have an account?" text and Login button to form layout
        form_layout.addSpacing(15)  
        # Add "Already have an account?" text
        forgot_label = QLabel('Did you forget your password?')
        forgot_label.setStyleSheet('color: white; font-size: 12px;')
        forgot_label.setAlignment(Qt.AlignCenter)

        form_layout.addWidget(forgot_label)
        form_layout.addWidget(forgot_password)
        form_layout.setAlignment(forgot_password, Qt.AlignCenter)

        # Main Layout
        main_layout = QVBoxLayout()
        main_layout.setSpacing(10)  # Reduce spacing between title and form
        main_layout.addWidget(title_label)
        main_layout.addLayout(form_layout)

        self.setLayout(main_layout)

    def handle_login(self):
        email = self.email_input.text()
        password = self.password_input.text()

        # Check if fields are filled
        if not email or not password:
            QMessageBox.warning(self, "Input Error", "Email and Password are required.")
            return

        try:
            connection = create_connection()
            cursor = connection.cursor()
            cursor.execute("SELECT * FROM userlist WHERE email = %s AND password = %s", (email, password))
            result = cursor.fetchone()

            if result:
                QMessageBox.information(self, "Success", "Login successful!")
                
                thread = threading.Thread(target=self.execute_notebook)
                thread.start()

                # Proceed to the next part of your application
            else:
                QMessageBox.warning(self, "Login Error", "Invalid email or password.")

        except mysql.connector.Error as error:
            QMessageBox.critical(self, "Database Error", f"Failed to query MySQL: {error}")
        finally:
            cursor.close()
            connection.close()

    def execute_notebook(self):
        subprocess.run(['jupyter', 'nbconvert', '--execute', '--to', 'notebook', '--inplace', 'code_code.ipynb'], check=True)
#         # Path to the .ipynb file
#         notebook_path = r"C:\Users\mahnoor\Documents\Python Scripts\FYProject\code_code.ipynb"  # Update with your actual path
        
#         # Use nbconvert to run the notebook
#         command = [
#             'jupyter', 'nbconvert', 
#             '--to', 'notebook', 
#             '--execute', 
#             '--inplace',  # Overwrites the existing notebook with the output
#             notebook_path
#         ]
        
#         # Run the command
#         subprocess.run(command)

#         # Optionally, open the notebook after execution
#         os.startfile(notebook_path)
        
    def clear_fields(self):
        self.email_input.clear()
        self.password_input.clear()
    
    def reset_password(self):
        email, ok = QInputDialog.getText(self, 'Reset Password', 'Enter your email:')
        if ok and email:
            if send_reset_email(email):
                QMessageBox.information(self, "Success", "Reset email sent! Check your inbox.")
                self.close()  # Close the signup form
                self.passkey_window = passkeyForm()  # Assuming LoginForm is the class for the login page
                self.passkey_window.show()  # Show the login window

            else:
                QMessageBox.warning(self, "Error", "Failed to send reset email. Please try again.")

class passkeyForm(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Pass Key Form')
        self.setGeometry(200, 200, 600, 400)  # Window size
        self.setStyleSheet("background-color: #2D2D2D;")  # Dark background

        # Font for title
        title_font = QFont('Georgia', 28, QFont.Bold | QFont.StyleItalic)  # New font style for the title
        
        # Viz Optilytics Title
        title_label = QLabel('Viz Optilytics')
        title_label.setFont(title_font)
        title_label.setStyleSheet('color: #E5B131;')  # Gold color
        title_label.setAlignment(Qt.AlignCenter)

        # Layout for form fields
        form_layout = QVBoxLayout()
        form_layout.setContentsMargins(40, 10, 40, 20)  # Set margins around the window (left, top, right, bottom)

        # Create a function to add each row (label + input field)
        def add_row(label_text, placeholder_text):
            # Horizontal layout for label and field
            row_layout = QHBoxLayout()

            # Label
            label = QLabel(label_text)
            label.setAlignment(Qt.AlignCenter)
            label.setStyleSheet("""
                
                color: white;
                border-radius: 10px;
                padding: 10px 20px;
                font-size: 14px;
                font-weight: bold;
            """)
            label.setFixedWidth(160)  # Increased width for label box to prevent text overflow

            # Input field
            input_field = QLineEdit()
            input_field.setPlaceholderText(placeholder_text)
            input_field.setStyleSheet("""
                background-color: #606060;
                color: white;
                border-radius: 10px;
                padding: 10px;
            """)
            input_field.setFixedHeight(40)

            # Add spacing between label and input
            row_layout.addWidget(label)
            row_layout.addSpacing(20)  # Add space between label and input field
            row_layout.addWidget(input_field)

            # Add the row layout to the main form layout
            form_layout.addLayout(row_layout)
            return input_field

        # Create each row
        self.email_input = add_row('Email', 'Testing@gmail.com')
        self.code_input = add_row('Code', '123456')
        
        # Register Button
        verify_button = QPushButton('verify')
        verify_button.setStyleSheet("""
            QPushButton {
                background-color: #00A74D;  /* Green color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #009945;
            }
            QPushButton:pressed {
                background-color: #008838;
            }
        """)
        verify_button.setFixedWidth(250)  # Increase button width to match input fields
        verify_button.clicked.connect(self.handle_code_verification)
        
        form_layout.addSpacing(30)  # Adds space before the Register button
        
        # Add Register button at the end
        form_layout.addWidget(verify_button)
        form_layout.setAlignment(verify_button, Qt.AlignCenter)
        
        form_layout.addSpacing(15)  
        # Add "Already have an account?" text
        resend_label = QLabel('Resend Verification code?')
        resend_label.setStyleSheet('color: white; font-size: 12px;')
        resend_label.setAlignment(Qt.AlignCenter)

        form_layout.addWidget(resend_label)
        # Login Button
        self.resend_button = QPushButton('resend code')  # Correctly store the button as an instance variable
        self.resend_button.setStyleSheet("""
            QPushButton {
                background-color: #d40d0d;  /* Red color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #C40C0C;
            }
            QPushButton:pressed {
                background-color: #b50b0b;
            }
        """)
        self.resend_button.setFixedWidth(250)
        self.resend_button.clicked.connect(self.handle_resend)  # Connect the signal properly

        form_layout.addWidget(self.resend_button)
        form_layout.setAlignment(self.resend_button, Qt.AlignCenter)

        # Main Layout
        main_layout = QVBoxLayout()
        main_layout.setSpacing(10)  # Reduce spacing between title and form
        main_layout.addWidget(title_label)
        main_layout.addLayout(form_layout)

        self.setLayout(main_layout)

    def handle_resend(self):
        email, ok = QInputDialog.getText(self, 'Reset Password', 'Enter your email:')
        if email:  # Only resend if an email is provided
            success = send_reset_email(email)
            if success:
                QMessageBox.information(self, "Success", "Reset code has been resent to your email.")
            else:
                QMessageBox.critical(self, "Failure", "Failed to resend the reset code. Please try again later.")
        
    def handle_code_verification(self):
        email = self.email_input.text()
        entered_code = self.code_input.text()

        if verify_reset_code(email, entered_code):
            # Code matches; open the reset password form
            self.close()  # Close the signup form
            self.reset_window = ResetForm()  # Assuming LoginForm is the class for the login page
            self.reset_window.show()  # Show the login window
        else:
            QMessageBox.warning(self, "Error", "Unable to verify the reset code.")

        
class ResetForm(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Reset Password Form')
        self.setGeometry(200, 200, 600, 400)  # Window size
        self.setStyleSheet("background-color: #2D2D2D;")  # Dark background

        # Font for title
        title_font = QFont('Georgia', 28, QFont.Bold | QFont.StyleItalic)  # New font style for the title
        
        # Viz Optilytics Title
        title_label = QLabel('Viz Optilytics')
        title_label.setFont(title_font)
        title_label.setStyleSheet('color: #E5B131;')  # Gold color
        title_label.setAlignment(Qt.AlignCenter)

        # Layout for form fields
        form_layout = QVBoxLayout()
        form_layout.setContentsMargins(40, 10, 40, 20)  # Set margins around the window (left, top, right, bottom)

        # Create a function to add each row (label + input field)
        def add_row(label_text, placeholder_text):
            # Horizontal layout for label and field
            row_layout = QHBoxLayout()

            # Label
            label = QLabel(label_text)
            label.setAlignment(Qt.AlignCenter)
            label.setStyleSheet("""
                
                color: white;
                border-radius: 10px;
                padding: 10px 20px;
                font-size: 14px;
                font-weight: bold;
            """)
            label.setFixedWidth(160)  # Increased width for label box to prevent text overflow

            # Input field
            input_field = QLineEdit()
            input_field.setPlaceholderText(placeholder_text)
            input_field.setStyleSheet("""
                background-color: #606060;
                color: white;
                border-radius: 10px;
                padding: 10px;
            """)
            input_field.setFixedHeight(40)

            # Add spacing between label and input
            row_layout.addWidget(label)
            row_layout.addSpacing(20)  # Add space between label and input field
            row_layout.addWidget(input_field)

            # Add the row layout to the main form layout
            form_layout.addLayout(row_layout)
            return input_field

        # Create each row
        self.email_input = add_row('Email', 'Testing@gmail.com')
        self.password_input = add_row('Password', '******')
        self.password_input.setEchoMode(QLineEdit.Password)
        self.verify_password_input = add_row('Verify Password', '******')
        self.verify_password_input.setEchoMode(QLineEdit.Password)

        # Add some vertical spacing before the Register button
        form_layout.addSpacing(30)  # Adds space before the Register button

        # Register Button
        register_button = QPushButton('Reset Password')
        register_button.setStyleSheet("""
            QPushButton {
                background-color: #00A74D;  /* Green color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #009945;
            }
            QPushButton:pressed {
                background-color: #008838;
            }
        """)
        register_button.setFixedWidth(250)  # Increase button width to match input fields
        register_button.clicked.connect(self.handle_reset_password)

        # Add Register button at the end
        form_layout.addWidget(register_button)
        form_layout.setAlignment(register_button, Qt.AlignCenter)

        form_layout.addSpacing(12)  
        # Add "Already have an account?" text
        already_account_label = QLabel('Back To Login?')
        already_account_label.setStyleSheet('color: white; font-size: 12px;')
        already_account_label.setAlignment(Qt.AlignCenter)

        # Login Button
        self.login_button = QPushButton('Login Now')  # Correctly store the button as an instance variable
        self.login_button.setStyleSheet("""
            QPushButton {
                background-color: #d40d0d;  /* Red color */
                color: white;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #C40C0C;
            }
            QPushButton:pressed {
                background-color: #b50b0b;
            }
        """)
        self.login_button.setFixedWidth(250)
        self.login_button.clicked.connect(self.open_login)  # Connect the signal properly

        # Add "Already have an account?" text and Login button to form layout
        form_layout.addSpacing(15)  # Add some space before the "Already have an account?" text
        form_layout.addWidget(already_account_label)
        form_layout.setAlignment(already_account_label, Qt.AlignCenter)
        form_layout.addWidget(self.login_button)  # Add self.login_button here
        form_layout.setAlignment(self.login_button, Qt.AlignCenter)

        # Main Layout
        main_layout = QVBoxLayout()
        main_layout.setSpacing(10)  # Reduce spacing between title and form
        main_layout.addWidget(title_label)
        main_layout.addLayout(form_layout)

        self.setLayout(main_layout)

    def open_login(self):
        self.close()  # Close the signup form
        self.login_window = LoginForm()  # Assuming LoginForm is the class for the login page
        self.login_window.show()  # Show the login window

    def handle_reset_password(self):
        email = self.email_input.text()
        password = self.password_input.text()
        verify_password = self.verify_password_input.text()

        # Check if all fields are filled
        if not email or not password or not verify_password:
            QMessageBox.warning(self, "Input Error", "All fields are required.")
            return

        # Validate email, username, and password
        if not self.validate_email(email):
            QMessageBox.warning(self, "Invalid Email", "Please enter a valid email address.")
            return

        if not self.validate_password(password):
            QMessageBox.warning(self, "Invalid Password", "Password does not meet the criteria.")
            return

        # Check if password and verify password match
        if password != verify_password:
            QMessageBox.warning(self, "Password Error", "Passwords do not match.")
            return

        try:
            connection = create_connection()
            cursor = connection.cursor()

            # Check if email or username already exists
            cursor.execute("SELECT * FROM userlist WHERE email = %s", (email,))
            result = cursor.fetchone()

            if not result:
                QMessageBox.warning(self, "Error", "Email does not exist.")
                return

            # Insert into database
            print(email)
            print(password)
            query = "UPDATE userlist SET password = %s WHERE email = %s"
            cursor.execute(query, (password, email))
            connection.commit()
            QMessageBox.information(self, "Success", "Password reset successfully!")
            self.reset_clear_fields()

        except mysql.connector.Error as error:
            QMessageBox.critical(self, "Database Error", f"Failed to insert data into MySQL: {error}")
        finally:
            if cursor:
                cursor.close()
            if connection:
                connection.close()

    def reset_clear_fields(self):
        self.email_input.clear()
        self.password_input.clear()
        self.verify_password_input.clear()
        
    def clear_fields(self):
        self.email_input.clear()
        self.username_input.clear()
        self.password_input.clear()
        self.verify_password_input.clear()

    def validate_email(self, email):
        pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
        return re.match(pattern, email)

    def validate_password(self, password):
        if len(password) < 8:
            return False
        if not re.search(r"[A-Z]", password):
            return False
        if not re.search(r"[a-z]", password):
            return False
        if not re.search(r"[0-9]", password):
            return False
        if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", password):
            return False
        return True

    def validate_username(self, username):
        if len(username) < 3 or len(username) > 15:
            return False
        if not re.match(r'^\w+$', username):  # Only letters, numbers, and underscores allowed
            return False
        return True

        
# Run the application
if __name__ == '__main__':
    app = QApplication([])
    window = SignupForm()
    window.show()
    app.exec_()


Connection successful
Failed to send email: {'Mahnoor1': (553, b'5.1.3 The recipient address <Mahnoor1> is not a valid RFC 5321 address. For\n5.1.3 more information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. 5b1f17b1804b1-433b4643100sm139750965e9.39 - gsmtp')}
Connection successful
Connection successful
Executing query: UPDATE userlist SET reset_code = %s, code_created_at = %s WHERE email = %s with values: 263518, 2024-11-24 02:07:23.821287, yasmeen@gmail
